In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
pd.set_option('mode.chained_assignment', None)
def my_formatter(x):
    if x < 0:
        return '{:,.2f}'.format(-x)
    return '{:,.2f}'.format(x)
pd.options.display.float_format = my_formatter
import calendar

## 藝感預算處理

In [2]:
df = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\2022_forecast_藝感_Response.xlsx", None)

In [3]:
df.keys()

dict_keys(['2020出貨量', '2021出貨量', '2022 Forecast'])

In [5]:
result = df['2022 Forecast']

In [12]:
result = pd.DataFrame(result.groupby(['INPAQ PN', 'Month'])[[' Qty. ']].sum().unstack('Month').to_records())

In [24]:
month_name = [calendar.month_name[i] for i in range(1, 13)]

In [25]:
month_name.insert(0, 'INPAQ PN')

In [26]:
result.columns = month_name

In [28]:
result.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\2022_forecast_藝感_sorted.xlsx", index=False)

## 信昌預算處理

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", '出貨明細')

In [ ]:
pdc = df[(df['Group'] == '信昌') & (df['幣別'] == 'NTD') & (df['狀態'] != '雜項訂單')]

In [ ]:
pdc_price = pdc.drop_duplicates(['品名', '客戶料號'], keep='last')

In [ ]:
pdc_result = pdc_price[['品名', '單價']]

In [ ]:
pdc_result.to_excel(r"C:\Users\kaihsu\Desktop\pdc_price.xlsx", index=False)

## MT-SYSTEM預算處理

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\2022_forecast_MT-SYSTEM_Response.xlsx", None)

In [ ]:
df.keys()

In [ ]:
component = df['2022 forecast Component']
rf = df['2022 forecast MT-System RF']

In [ ]:
month_name = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
rf['Month'] = pd.Categorical(rf['Month'], ordered=True, categories=month_name)
component['Month'] = pd.Categorical(component['Month'], ordered=True, categories=month_name)

In [ ]:
def convert_table_qty(data):
    result = data.groupby(['P/N', 'Month'])[['Qty.']].sum().unstack()
    result = pd.DataFrame(result.to_records())
    return result

In [ ]:
com_result = convert_table_qty(component)
rf_result = convert_table_qty(rf)

In [ ]:
com_result.to_excel(r'C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM元件預算.xlsx', index=False)
rf_result.to_excel(r'C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM天線預算.xlsx', index=False)

In [88]:
# MT-SYSTEM目前價格處理
mt_price = df[(df['Group'] == 'MT-SYSTEM') & (df['預交年份'].isin([2022, 2021]))]

In [90]:
mt_price = mt_price.drop_duplicates(subset=['品名', '單價'], keep='last')

In [91]:
mt_price_dict = dict(zip(mt_price['品名'], mt_price['單價']))

In [92]:
mt_sorted_com = df[(df['Group'] == 'MT-SYSTEM') & (df['預交年份'] == 2022) & ~(df['BU'].isin(['RF1', 'RF2']))]

In [100]:
answer_mt_com = pd.DataFrame(mt_sorted_com.groupby(['品名', mt_sorted_com['預交日期'].dt.to_period('M')])[['數量']].sum().unstack('預交日期').to_records())

In [103]:
answer_mt_com.columns = ['PN', '1', '2', '3', '4', '6', '7', '8', '9']

In [104]:
answer_mt_com['單價'] = answer_mt_com['PN'].map(mt_price_dict)

In [106]:
answer_mt_com.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM元件預算_from_OPO.xlsx", index=False)

In [107]:
origin_mt_com = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM元件預算.xlsx")

In [112]:
origin_mt_com['單價'] = origin_mt_com['P/N'].map(mt_price_dict)

In [113]:
origin_mt_com.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM元件預算.xlsx", index=False)

In [115]:
origin_mt_rf = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM天線預算.xlsx")

In [117]:
origin_mt_rf['單價'] = origin_mt_rf['P/N'].map(mt_price_dict)

In [119]:
origin_mt_rf.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\MT-SYSTEM天線預算.xlsx", index=False)

## Teltonika數據轉置

In [ ]:
month_name = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
month_dict = {}

In [ ]:
for x, y in enumerate(month_name):
    month_dict.setdefault(y, x+1)

In [ ]:
df = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\2022_forecast_Teltonika_Response.xlsx", None)

In [ ]:
component = df['2022 Forecast Component']
antenna = df['2022 Forecast Antenna']

In [ ]:
def convert_table_qty(data):
    data = data[data['Year'] == 2022]
    data['Month'] = data['Month'].map(month_dict)
    result = data.groupby(['INPAQ PN', 'Month'])[[' Qty. ']].sum().unstack()
    result = pd.DataFrame(result.to_records())
    return result

In [ ]:
com_result_qty = convert_table_qty(component)

In [ ]:
ant_result_qty = convert_table_qty(antenna)

In [ ]:
com_result_qty.to_excel(r'C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\TELTONIKA元件預算.xlsx', index=False)
ant_result_qty.to_excel(r'C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\TELTONIKA天線預算.xlsx', index=False)

In [29]:
# TELTONIKA 2021年下的2022年訂單整理
df = pd.read_excel(r"C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx", sheet_name='出貨明細')

In [39]:
# 價格表做成字典檔
for_price = df[(df['Group'] == 'TELTONIKA') & (df['預交年份'] == 2022)]

In [42]:
price = for_price.drop_duplicates(subset=['品名', '單價'], keep='last')

In [47]:
price_dict = dict(zip(price['品名'], price['單價']))

In [53]:
df['BU'].unique()

array([nan, 'PC', 'WIP', 'AIP', 'EMC', 'MAG', 'RF1', 'RF2', 'ODM'],
      dtype=object)

In [64]:
sorted_com = df[(df['Group'] == 'TELTONIKA') & (df['預交年份'] == 2022) & ~(df['BU'].isin(['RF1', 'RF2']))]

In [71]:
answer = pd.DataFrame(sorted_com.groupby(['品名', sorted_com['預交日期'].dt.to_period('M')])[['數量']].sum().unstack('預交日期').to_records())

In [75]:
answer.columns = ['PN', '1', '2', '3', '4', '5','6', '7', '8', '9', '11', '12']

In [78]:
answer.insert(10, '10', '')

In [80]:
answer['單價'] = answer['PN'].map(price_dict)

In [83]:
answer.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\TELTONIKA元件預算_from_OPO.xlsx", index=False)

In [120]:
origin_tel_rf = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\TELTONIKA天線預算.xlsx")

In [123]:
origin_tel_rf['單價'] = origin_tel_rf['INPAQ PN'].map(price_dict)

In [124]:
origin_tel_rf.to_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\整理完的數據\TELTONIKA天線預算.xlsx", index=False)